In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch < 15:
        return 0.0005
    elif epoch < 30:
        return 0.0001
    elif epoch < 45:
        return 0.00005
    else:
        return 0.00001

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

     # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [5]:
def kfold_train(fold_cnt=3,rnd=99):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.77261, saving model to best_m.h5
 - 8s - loss: 0.6154 - acc: 0.7231 - val_loss: 0.7726 - val_acc: 0.4539
Epoch 2/120
Epoch 00002: val_loss improved from 0.77261 to 0.73358, saving model to best_m.h5
 - 4s - loss: 0.4282 - acc: 0.7964 - val_loss: 0.7336 - val_acc: 0.5511
Epoch 3/120
Epoch 00003: val_loss improved from 0.73358 to 0.66839, saving model to best_m.h5
 - 4s - loss: 0.3998 - acc: 0.8122 - val_loss: 0.6684 - val_acc: 0.7132
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.3240 - acc: 0.8516 - val_loss: 1.0834 - val_acc: 0.6858
Epoch 5/120
Epoch 00005: val_loss improved from 0.66839 to 0.65868, saving model to best_m.h5
 - 4s - loss: 0.3159 - acc: 0.8650 - val_loss: 0.6587 - val_acc: 0.7731
Epoch 6/120
Epoch 00006: val_loss improved from 0.65868 to 0.38624, saving model to best_m.h5
 - 4s - loss: 0.3259 - acc: 0.8616 - val_loss: 0.3862 - val_acc: 0.7930
Epoch 7/120
Epoch 00007: val_loss did not improve
 -

Epoch 10/120
Epoch 00010: val_loss did not improve
 - 4s - loss: 0.3212 - acc: 0.8645 - val_loss: 0.9454 - val_acc: 0.7232
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 4s - loss: 0.3658 - acc: 0.8589 - val_loss: 0.3535 - val_acc: 0.8529
Epoch 12/120
Epoch 00012: val_loss improved from 0.28290 to 0.19793, saving model to best_m.h5
 - 4s - loss: 0.2747 - acc: 0.8850 - val_loss: 0.1979 - val_acc: 0.9152
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 4s - loss: 0.2934 - acc: 0.8792 - val_loss: 0.3153 - val_acc: 0.8354
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 4s - loss: 0.3332 - acc: 0.8784 - val_loss: 0.3066 - val_acc: 0.8653
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 4s - loss: 0.3259 - acc: 0.8568 - val_loss: 0.7065 - val_acc: 0.7257
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 4s - loss: 0.2541 - acc: 0.8950 - val_loss: 0.2514 - val_acc: 0.8928
Epoch 17/120
Epoch 00017: val_loss improved from 0.19793 to 0.19383, saving model to best_m.h5


Epoch 20/120
Epoch 00020: val_loss did not improve
 - 4s - loss: 0.1849 - acc: 0.9283 - val_loss: 0.2320 - val_acc: 0.9152
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 4s - loss: 0.2176 - acc: 0.9125 - val_loss: 0.2318 - val_acc: 0.9077
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 4s - loss: 0.2136 - acc: 0.9200 - val_loss: 0.2222 - val_acc: 0.9102
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 4s - loss: 0.2170 - acc: 0.9083 - val_loss: 0.2553 - val_acc: 0.8978
Epoch 24/120
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2190 - acc: 0.9183 - val_loss: 0.2145 - val_acc: 0.9102
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.1979 - acc: 0.9242 - val_loss: 0.2568 - val_acc: 0.9027
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2227 - acc: 0.9089 - val_loss: 0.2481 - val_acc: 0.9052
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2695 - acc: 0.8937 - val_loss: 0.2431 - val_acc: 0.9102
Epoch 28/120
Epo

Epoch 47/120
Epoch 00047: val_loss did not improve
 - 4s - loss: 0.1389 - acc: 0.9414 - val_loss: 0.2544 - val_acc: 0.8903
Epoch 48/120
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.1444 - acc: 0.9416 - val_loss: 0.2514 - val_acc: 0.8928


In [6]:
with open('../features/incept_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug3_sub.csv', index=False)
# deep2 0.2110
# deep3 0.2119

0.211937121193
         id  is_iceberg
0  5941774d    0.045901
1  4023181e    0.894050
2  b20200e4    0.043035
3  e7f018bb    0.998357
4  4371c8c3    0.064622


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=999):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.70302, saving model to best_m.h5
 - 25s - loss: 1.1298 - acc: 0.5780 - val_loss: 0.7030 - val_acc: 0.4539
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5739 - acc: 0.6950 - val_loss: 1.2136 - val_acc: 0.4539
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.5754 - acc: 0.7136 - val_loss: 0.7909 - val_acc: 0.4564
Epoch 4/120
Epoch 00004: val_loss improved from 0.70302 to 0.53721, saving model to best_m.h5
 - 6s - loss: 0.4922 - acc: 0.7533 - val_loss: 0.5372 - val_acc: 0.7731
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 6s - loss: 0.5182 - acc: 0.7589 - val_loss: 0.5945 - val_acc: 0.5561
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.4717 - acc: 0.7506 - val_loss: 2.1159 - val_acc: 0.6459
Epoch 7/120
Epoch 00007: val_loss improved from 0.53721 to 0.37618, saving model to best_m.h5
 - 6s - loss: 0.4528 - acc: 0.7897 - val_loss: 0.3762 - val_acc: 0.8404
Epoch 8/120
Epoch

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 6s - loss: 0.2844 - acc: 0.8747 - val_loss: 0.1886 - val_acc: 0.9227
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 6s - loss: 0.2367 - acc: 0.8950 - val_loss: 0.1886 - val_acc: 0.9302
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 6s - loss: 0.2491 - acc: 0.8914 - val_loss: 0.1878 - val_acc: 0.9302
Epoch 65/120
Epoch 00065: val_loss improved from 0.18336 to 0.18244, saving model to best_m.h5
 - 6s - loss: 0.2545 - acc: 0.8992 - val_loss: 0.1824 - val_acc: 0.9302
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 6s - loss: 0.2465 - acc: 0.8964 - val_loss: 0.1855 - val_acc: 0.9327
Epoch 67/120
Epoch 00067: val_loss improved from 0.18244 to 0.18222, saving model to best_m.h5
 - 6s - loss: 0.2302 - acc: 0.9075 - val_loss: 0.1822 - val_acc: 0.9302
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.2463 - acc: 0.8850 - val_loss: 0.1825 - val_acc: 0.9302
Epoch 69/120
Epoch 00069: val_loss improved from 0.

Epoch 4/120
Epoch 00004: val_loss improved from 0.69139 to 0.47139, saving model to best_m.h5
 - 6s - loss: 0.4062 - acc: 0.8083 - val_loss: 0.4714 - val_acc: 0.7456
Epoch 5/120
Epoch 00005: val_loss improved from 0.47139 to 0.37567, saving model to best_m.h5
 - 6s - loss: 0.3903 - acc: 0.8264 - val_loss: 0.3757 - val_acc: 0.8279
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.3879 - acc: 0.8231 - val_loss: 0.4562 - val_acc: 0.8105
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 6s - loss: 0.3955 - acc: 0.8153 - val_loss: 0.7176 - val_acc: 0.7257
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 6s - loss: 0.3752 - acc: 0.8150 - val_loss: 0.8228 - val_acc: 0.7282
Epoch 9/120
Epoch 00009: val_loss did not improve
 - 6s - loss: 0.3542 - acc: 0.8433 - val_loss: 0.4108 - val_acc: 0.7855
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 7s - loss: 0.3808 - acc: 0.8354 - val_loss: 0.4093 - val_acc: 0.7581
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 6s 

 - 6s - loss: 0.2177 - acc: 0.9087 - val_loss: 0.2548 - val_acc: 0.8878
Epoch 66/120
Epoch 00066: val_loss improved from 0.25477 to 0.25302, saving model to best_m.h5
 - 6s - loss: 0.1938 - acc: 0.9075 - val_loss: 0.2530 - val_acc: 0.8903
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.2119 - acc: 0.9173 - val_loss: 0.2552 - val_acc: 0.8903
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.2248 - acc: 0.9081 - val_loss: 0.2556 - val_acc: 0.8928
Epoch 69/120
Epoch 00069: val_loss improved from 0.25302 to 0.25149, saving model to best_m.h5
 - 6s - loss: 0.2000 - acc: 0.9033 - val_loss: 0.2515 - val_acc: 0.8878
Epoch 70/120
Epoch 00070: val_loss improved from 0.25149 to 0.25132, saving model to best_m.h5
 - 6s - loss: 0.1926 - acc: 0.9181 - val_loss: 0.2513 - val_acc: 0.8878
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 6s - loss: 0.2044 - acc: 0.9175 - val_loss: 0.2542 - val_acc: 0.8903
Epoch 72/120
Epoch 00072: val_loss improved from 0.25132 t

Epoch 00007: val_loss did not improve
 - 6s - loss: 0.5187 - acc: 0.7541 - val_loss: 0.4473 - val_acc: 0.8130
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 6s - loss: 0.4556 - acc: 0.8056 - val_loss: 0.4932 - val_acc: 0.7955
Epoch 9/120
Epoch 00009: val_loss improved from 0.39263 to 0.37587, saving model to best_m.h5
 - 6s - loss: 0.4729 - acc: 0.7883 - val_loss: 0.3759 - val_acc: 0.8279
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 6s - loss: 0.5722 - acc: 0.7462 - val_loss: 0.5186 - val_acc: 0.7057
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 6s - loss: 0.4863 - acc: 0.8022 - val_loss: 1.0953 - val_acc: 0.6808
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 6s - loss: 0.4888 - acc: 0.7856 - val_loss: 0.5878 - val_acc: 0.7930
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 6s - loss: 0.4528 - acc: 0.8333 - val_loss: 0.5598 - val_acc: 0.7431
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 6s - loss: 0.5723 - acc: 0.7643 - val_loss: 1.2521 - val

Epoch 69/120
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.2435 - acc: 0.9092 - val_loss: 0.2883 - val_acc: 0.8728
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 6s - loss: 0.2566 - acc: 0.9023 - val_loss: 0.2963 - val_acc: 0.8703
Epoch 71/120
Epoch 00071: val_loss improved from 0.28268 to 0.27614, saving model to best_m.h5
 - 6s - loss: 0.2446 - acc: 0.9242 - val_loss: 0.2761 - val_acc: 0.8853
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 6s - loss: 0.2354 - acc: 0.9158 - val_loss: 0.2862 - val_acc: 0.8753
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 6s - loss: 0.2819 - acc: 0.8970 - val_loss: 0.2809 - val_acc: 0.8728
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 6s - loss: 0.2516 - acc: 0.9067 - val_loss: 0.2796 - val_acc: 0.8778
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 6s - loss: 0.2411 - acc: 0.9067 - val_loss: 0.2804 - val_acc: 0.8753
Epoch 76/120
Epoch 00076: val_loss improved from 0.27614 to 0.27408, saving model to best_m.h5


Epoch 00011: val_loss improved from 0.38447 to 0.34593, saving model to best_m.h5
 - 6s - loss: 0.4426 - acc: 0.7922 - val_loss: 0.3459 - val_acc: 0.8180
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 6s - loss: 0.4368 - acc: 0.7839 - val_loss: 0.9932 - val_acc: 0.7382
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 6s - loss: 0.3760 - acc: 0.8225 - val_loss: 0.7967 - val_acc: 0.7257
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 6s - loss: 0.4495 - acc: 0.7222 - val_loss: 0.3774 - val_acc: 0.8254
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 6s - loss: 0.4032 - acc: 0.8258 - val_loss: 0.6112 - val_acc: 0.7756
Epoch 16/120
Epoch 00016: val_loss improved from 0.34593 to 0.31056, saving model to best_m.h5
 - 6s - loss: 0.3326 - acc: 0.8541 - val_loss: 0.3106 - val_acc: 0.8429
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 6s - loss: 0.3137 - acc: 0.8625 - val_loss: 0.3346 - val_acc: 0.8279
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 6s - loss:

Epoch 74/120
Epoch 00074: val_loss did not improve
 - 6s - loss: 0.1956 - acc: 0.9183 - val_loss: 0.2124 - val_acc: 0.9177
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 6s - loss: 0.2115 - acc: 0.9125 - val_loss: 0.2140 - val_acc: 0.9202
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 6s - loss: 0.2132 - acc: 0.9175 - val_loss: 0.2152 - val_acc: 0.9202
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 6s - loss: 0.2172 - acc: 0.9031 - val_loss: 0.2178 - val_acc: 0.9152


In [9]:
with open('../features/resnet_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug3_sub.csv', index=False)
# deep2 0.227
# deep3 0.223

0.22384510401
         id  is_iceberg
0  5941774d    0.103475
1  4023181e    0.687498
2  b20200e4    0.082042
3  e7f018bb    0.964287
4  4371c8c3    0.032139
